# Cost Analysis of a facility

Cost is defined as $ c_{i,j} = \sum_{(k,l)} \sum_{p} \frac{w_{p,(k,l)}}{d_{(i,j),(k,l)}} $ 

$w_{p,(k,l)} $ is the weight of a product to point $(k,l)$ 

$d_{(i,j),(k,l)}$ is the distance from $(i,j)$ to $(k,l)$

To derive the weight, it will be calculated from product of relative volume, and CORELAP score

In [1]:
sf = 1 # Facility scaling factor
## Create facility
Wfacility = sum([205 233 208 206 71 265 265 281 286 282 289 289 288 285 276 395]) # width of facility [inches]
Lfacility = sum([328 326 326 372 394 138]) # length of facility [inches]

Wfacility = div(Wfacility, sf) # put in terms of scaling factor
Lfacility = div(Lfacility, sf); 

At this point the weights will be calculated using the scores given by a CORELAP, the sum across the row. This will be multiplied by the weekly volume at these stations. These values will be constant throughout all layouts.

In [2]:
# CORELAP Scores
shredCOR = [6 6 6 6 5 5 6 5 2 2 0 6 5 4 4 4 2 4 2]
blCOR =    [5 5 5 5 3 4 4 4 0 6 2 2 6 5 5 2 2 6 2]
omniCOR =  [2 2 2 2 2 2 2 2 2 2 4 5 6 2 2 0 2 6 2]
prntCOR =  [2 6 2 2 6 2 0 2 4 5 6 4 3 4 5 2 2 5 2]
tvCOR =    [6 2 2 2 6 0 2 2 4 5 5 4 3 4 5 2 5 6 3]
miscCOR =  [2 2 6 2 6 2 2 0 4 5 5 4 3 5 5 2 2 5 5]

# Volume
shredV = 300 * 500 # number of gaylords multiplied by average weight of a gaylord
blV =    37 * 500 
omniV =  300 * 500
prntV =  55000
tvV = prntV
miscV =  10000
totalV = sum([shredV, blV, omniV, prntV, tvV, miscV])

# weights
shredW = sum(shredCOR) * shredV#/totalV
blW = sum(blCOR) * blV#/totalV
omniW = sum(omniCOR) * omniV#/totalV
prntW = sum(prntCOR) * prntV#/totalV
tvW = sum(tvCOR) * tvV#/totalV
miscW = sum(miscCOR) * miscV#/totalV
;

In [3]:
## Create Distance Matrices to important points
function getdist(Lfacility, Wfacility, point)
    distances = zeros(Lfacility, Wfacility)  
    for j=1:Lfacility
        for k=1:Wfacility
            if j == point[1] && k == point[2]
                distances[j,k] = 1
            else
                distances[j,k] = sqrt((point[1]-j)^2 + (point[2]-k)^2)
            end
        end
    end
    return distances
end

getdist (generic function with 1 method)

The variables among the different layouts will be distance, and will be split among 3 designs. 
1. No change
2. Just change workstation
3. Complete alteration

In [11]:
## Set Locations for no change to facility. 
# Shredder
shrdRow = div(sum([328 326]),sf)
shrdCol = div(sum([205 233 233 277 265 265]), sf)
shredder = [shrdRow shrdCol]

# Bailer
blRow = div(sum( [328 326 326 (336/2)] ), sf)
blCol = div( sum([205 233 233 277 265 265 281 286 282 289] ), sf)
bailer = [blRow blCol]

# Omnisource
omRow = Lfacility 
omCol = 1
omnisource = [omRow omCol]

# Printer Station
prntRow = div( sum([328 326 326 372]), sf)
prntCol = div( sum([205 233 233 277 265 265 281]), sf)
printer = [prntRow prntCol]

# TV Station
tvRow = div( sum([328 326 326 372]), sf)
tvCol = div( sum([205 233 233 277 265]), sf)
tv = [tvRow, tvCol]

# Misc Station
miscRow = div( sum([328 326 326 372]), sf)
miscCol = div( sum([205 233 233 277 265 265 281 286 282]), sf)
misc = [miscRow miscCol];

In [12]:
shredDist = getdist(Lfacility, Wfacility, shredder)
blDist = getdist(Lfacility, Wfacility, bailer)
omniDist = getdist(Lfacility, Wfacility, omnisource)
prntDist = getdist(Lfacility, Wfacility, printer)
tvDist = getdist(Lfacility, Wfacility, tv)
miscDist = getdist(Lfacility, Wfacility, misc)
;

In [13]:
cost = 1./shredDist .* shredW
cost += 1./blDist .* blW
cost += 1./omniDist .* omniW
cost += 1./prntDist .* prntW
cost += 1./tvDist .* tvW
cost += 1./miscDist .* miscW
;

In [14]:
using FileIO
using Images
using ImageMagick
using DataFrames
save("costsNoChange.png",grayim(log(cost)/maximum(log(cost))))
writecsv("costNoChange.csv", cost)

In [15]:
## Set Locations for flipping staging and workstation
# Shredder
shrdRow = div(sum([328 326]),sf)
shrdCol = div(sum([205 233 233 277 265 265]), sf)
shredder = [shrdRow shrdCol]

# Bailer
blRow = div(sum( [328 326 326 (336/2)] ), sf)
blCol = div( sum([205 233 233 277 265 265 281 286 282 289] ), sf)
bailer = [blRow blCol]

# Omnisource
omRow = Lfacility 
omCol = 1
omnisource = [omRow omCol]

# Printer Station
prntRow = div( sum([328 326 326]), sf)
prntCol = div( sum([205 233 233 277 265 265 281]), sf)
printer = [prntRow prntCol]

# TV Station
tvRow = div( sum([328 326 326]), sf)
tvCol = div( sum([205 233 233 277 265]), sf)
tv = [tvRow, tvCol]

# Misc Station
miscRow = div( sum([328 326 326]), sf)
miscCol = div( sum([205 233 233 277 265 265 281 286 282]), sf)
misc = [miscRow miscCol];


In [16]:
shredDist = getdist(Lfacility, Wfacility, shredder)
blDist = getdist(Lfacility, Wfacility, bailer)
omniDist = getdist(Lfacility, Wfacility, omnisource)
prntDist = getdist(Lfacility, Wfacility, printer)
tvDist = getdist(Lfacility, Wfacility, tv)
miscDist = getdist(Lfacility, Wfacility, misc)
;

In [17]:
cost = 1./shredDist .* shredW
cost += 1./blDist .* blW
cost += 1./omniDist .* omniW
cost += 1./prntDist .* prntW
cost += 1./tvDist .* tvW
cost += 1./miscDist .* miscW
;

In [18]:
using FileIO
using Images
using ImageMagick
using DataFrames
save("costsFlipWS.png",grayim(log(cost)/maximum(log(cost))))
writecsv("costFlipWS.csv", cost)

In [19]:
## Set Locations for complete alteration
# Shredder
shrdRow = div(sum([328 326]),sf)
shrdCol = div(sum([205 233 233 277 265 265]), sf)
shredder = [shrdRow shrdCol]

# Bailer
blRow = div(sum( [328 326 326 (336/2)] ), sf)
blCol = div( sum([205 233 233 277 265 265 281 286 282 289] ), sf)
bailer = [blRow blCol]

# Omnisource
omRow = Lfacility 
omCol = 1
omnisource = [omRow omCol]

# Printer Station
prntRow = div( sum([328 326 326]), sf)
prntCol = div( sum([205 233 233 277 265]), sf)
printer = [prntRow prntCol]

# TV Station
tvRow = div( sum([328 326/2]), sf)
tvCol = div( sum([205 233 265 277 265 281]), sf)
tv = [tvRow, tvCol]

# Misc Station
miscRow = div( sum([328 326 326]), sf)
miscCol = div( sum([205 233 208/2]), sf)
misc = [miscRow miscCol];


In [20]:
shredDist = getdist(Lfacility, Wfacility, shredder)
blDist = getdist(Lfacility, Wfacility, bailer)
omniDist = getdist(Lfacility, Wfacility, omnisource)
prntDist = getdist(Lfacility, Wfacility, printer)
tvDist = getdist(Lfacility, Wfacility, tv)
miscDist = getdist(Lfacility, Wfacility, misc)
;

In [21]:
cost = 1./shredDist .* shredW
cost += 1./blDist .* blW
cost += 1./omniDist .* omniW
cost += 1./prntDist .* prntW
cost += 1./tvDist .* tvW
cost += 1./miscDist .* miscW
;

In [22]:
using FileIO
using Images
using ImageMagick
using DataFrames
save("costsFullChange.png",grayim(log(cost)/maximum(log(cost))))
writecsv("costFullChange.csv", cost)